<a href="https://colab.research.google.com/github/krumeto/own_work/blob/master/Pneumonia_detection_in_x_ray_images_Final_Project_Krum_Arnaudov%2C_SoftUni_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pneumonia detection in x-ray images - training CNN without Transfer learning

### Autor: Krum Arnaudov for Deep Learning SoftUni - December 2018

## Abstract
With this study I would like to achieve or improve upon the current Kaggle Leaderboard results on a X-ray image dataset, by not using transfer learning, but rather training a convolutional neural network from scratch. I start from a standard basic model and improve it by using regularization techniques and data augmentation, achieving test accuracy, recall and precision on par with the leading Kaggle Kernel.

In [0]:
%matplotlib inline

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os, shutil
from pathlib import Path
import glob
from google.colab import drive
import cv2
from skimage.io import imread
from skimage.io import imshow


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from mlxtend.plotting import plot_confusion_matrix

In [0]:
# Set the numpy seed
np.random.seed(42)

# Set the random seed in tensorflow at graph level
tf.set_random_seed(42)

In [0]:
#Connect to Google Drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Introduction

Madical image analysis is a relatively old, but still evolving application field for machine and deep learning. For many medical topics, the models have achieved equal or higher level of success rate than the human experts. Due to the success of ML/DL in the field, it is a beloved topic for Kaggle competitions. 

For this work, I choose a dataset of X-Ray images, provided on Kaggle. The main reasons for my choice:
- Kaggle kernel results provide for a useful benchmarking;
- The dataset is relatively small and will allow for faster itteration and model tuning; 
- The little data poses a realistic challenge - large medical datasets would not always be easy to attain.
- Last but not least, it was an interesting starting point in medical imaging.

My goal for the current work is to train a relatively small Convolutional Neural Network from scratch and achieve results, comparable to the Kaggle leaderboard. As the top Kaggle kernels at the time of writing use transfer learning, achieving similar or better results from scratch would be rather satisfactory. 

I also use the opportunity to work on Google Colab, using its GPU Runtime. 

In [0]:
# The path to the base directory
base_dir = Path("/content/gdrive/My Drive/chest_xray")

# Path to train directory
train_dir = base_dir / 'train'

# Path to validation directory
val_dir = base_dir / 'val'

# Path to test directory
test_dir = base_dir / 'test'



In [0]:
print("Total number of images:", len(list(base_dir.rglob('*.jpeg')))-16) # -16 to show the original form of the datase

print("Total number of training images:", len(list(train_dir.rglob('*.jpeg')))- 16) # same as above

print("Total number of validation images:", len(list(val_dir.rglob('*.jpeg'))))

print("Total number of test images:", len(list(test_dir.rglob('*.jpeg'))))

The dataset as provided at Kaggle has a total of 5856 grayscale X-Ray images of 2 categories - Normal (healthy) images and Pneumonia ones, split in a test, validation and training set. It has a very small validation set - only 8 images per category.

While Kaggle competitors might be restricted to the small validation set, I believe that, considering the overall small dataset, working with cross-validation is the better choice. 

Therefore, I will add the validation set images to the training set and setup cross-validation when training the model. 

In [0]:
all_normal_images = 0
all_pneumonia_images = 0

for directory in [train_dir,test_dir]:
  normal_dir = directory / 'NORMAL'
  pneumonia_dir = directory / 'PNEUMONIA'
  all_normal_images += len(list(normal_dir.rglob('*.jpeg')))
  all_pneumonia_images += len(list(pneumonia_dir.rglob('*.jpeg')))
  print("Normal images in {} directory:".format(directory.name), len(list(normal_dir.rglob('*.jpeg'))))
  print("Pneumonia images in {} directory:".format(directory.name), len(list(pneumonia_dir.rglob('*.jpeg'))))
  
print("Total number of healthy images:", all_normal_images)
print("Total number of pneumonia images:", all_pneumonia_images)

The dataset is highly imbalanced towards the pneumonia images - 4273 vs. only 1583 healthy images. This will have to be accounted for when assessing the overall success of the model - accuracy will not be a good final metrics. 

Let us look at some of the actual images:

In [0]:
healthy_dir = train_dir / "NORMAL"
pneumonia_dir = train_dir / "PNEUMONIA"

# Get few samples for both the classes
pneumonia_samples = []
normal_samples = []

for pn_sample in pneumonia_dir.glob('*.jpeg'):
  pneumonia_samples.append(pn_sample)

for norm_sample in healthy_dir.glob('*.jpeg'):
  normal_samples.append(norm_sample)

samples = pneumonia_samples[:5] + normal_samples[:5]


# Plot the data 
f, ax = plt.subplots(2,5, figsize=(20,6.5))
for i in range(10):
    img = imread(samples[i])
    ax[i//5, i%5].imshow(img, cmap='gray')
    if i<5:
        ax[i//5, i%5].set_title("Pneumonia")
    else:
        ax[i//5, i%5].set_title("Normal")
    ax[i//5, i%5].axis('off')
    ax[i//5, i%5].set_aspect('auto')
plt.show()

"White where there should not be white. However, it is not as straightforward" - this is the description a junior medicine specialist gave me as a hint how pneumonia can be recognized. Looking at the images above, the description fits some very well, but some are definately confusing. 

## Base model

I will use the infrastructure suggested by Google's employee François Chollet in a Keras Blog article on CNNs and small datasets. 

I will start with a base model and add to it, depending on the initial results. 

To start with:
- Images will be re-shaped to 150/150/3 format, allowing for faster training
- 4 blocks of Convolution+Pooling layers will be followed by 2 Dense layers
- Total number of trainable params is 3,485,505 - rather modest, compared to the more than 104 millions for the Kaggle leaders.
- Loss function will be Binary Crossentropy - a popular choice for binary classification
- Optimizer will be Adam, with a very low learning rate of 0,0001. I will adjust the learning rate in case of slow learning.
- Metric is accuracy for the sake of consistency - all curent Kaggle Kernels use it despite the imbalanced dataset. However, my final evaluation will look at precision and recall.

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(64, activation='relu'))


model.add(Dense(1, activation='sigmoid'))


In [0]:
model.summary()

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(1e-4), #Setting a very low learning rate to start with
              metrics=['accuracy'])

I found Keras ImageDataGenerator extremely helpful, considering the structure of the dataset - separated in folders per category. 

The generator allows for several useful steps:
- K-fold cross-validation of 10% of the test data (522 images).
- resizing to the target size of images
- Data augmentation (I will use it down the road and not in the first version of the model)

In [0]:
batch_size = 16

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1) #setting the validation split percentage
#test_datagen = ImageDataGenerator(rescale=1./255)


# this is a generator that will read pictures found in
# subfolers of 'train/dir', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        train_dir,  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary', # since we use binary_crossentropy loss, we need binary labels
        subset='training')  # setting the training data

# this is a similar generator, for validation data
validation_generator = train_datagen.flow_from_directory(
        train_dir,  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary', # since we use binary_crossentropy loss, we need binary labels
        subset='validation') # setting the validation data

In [0]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=4710 // batch_size, # the generator needs to stop in time, therefore this setting
        epochs=30,
        validation_data=validation_generator,
        validation_steps=522 // batch_size)

model.save_weights('first_try.h5')

In [0]:
def model_results(NN_model, size):
  """
  A function to get Accuracy and Loss, based on the model and the size of the input
  """
  test_datagen = ImageDataGenerator(rescale=1./255)
  test_generator = test_datagen.flow_from_directory(directory = test_dir,
                                  target_size=(size, size), 
                                  batch_size=batch_size,
                                  class_mode='binary')
  
  scores = NN_model.evaluate_generator(test_generator,624)
  print("Test Accuracy = ", scores[1])
  print("Test Loss = ", scores[0])

In [0]:
def NN_visualization(model_history):
  """
  A function to visualize the Training and Validation accuracy and loss
  """
  acc = model_history.history['acc']
  val_acc = model_history.history['val_acc']
  loss = model_history.history['loss']
  val_loss = model_history.history['val_loss']

  epochs = range(len(acc))

  plt.plot(epochs, acc, 'bo', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()

  plt.figure()

  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()

In [0]:
NN_visualization(history)

In [0]:
model_results(model, 150)

These plots are characteristic of overfitting. The training accuracy increases almost linearly over time, until it reaches nearly 100%, while the validation accuracy stalls at 93-97%. The validation loss reaches its minimum after only five epochs then decays, while the training loss keeps decreasing linearly until it reaches nearly 0.

Still, accuracy on the test set is 76.7% which is a modest but not so bad of a start.

## Model with Dropout and L2 regularization

Because of the relatively few training samples, overfitting is going to be my number one concern. I will try adding Dropout layers and weight decay.

**Dropout** is an extremely effective and simple regularization technique by Srivastava et al. in Dropout: A Simple Way to Prevent Neural Networks from Overfitting. While training, dropout is implemented by only keeping a neuron active with some probability p (a hyperparameter), or setting it to zero otherwise.

**L2 regularization** is perhaps the most common form of regularization. It can be implemented by penalizing the squared magnitude of all parameters directly in the objective. 

In [0]:
model2 = Sequential()
model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model2.add(MaxPooling2D((2, 2)))

model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))

model2.add(Conv2D(128, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Dropout(0.5)) #Adding Dropout with p = 0.5

model2.add(Conv2D(128, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Dropout(0.5))#Adding Dropout with p = 0.5

model2.add(Flatten())
model2.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))) #Adding L2
model2.add(Dropout(0.5)) #Adding Dropout with p = 0.5
model2.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))#Adding L2


model2.add(Dense(1, activation='sigmoid'))

In [0]:
model2.summary()

In [0]:
model2.compile(loss='binary_crossentropy',
              optimizer=Adam(1e-4), #Retaining the very low learning rata
              metrics=['accuracy'])

In [0]:
history2 = model2.fit_generator(
        train_generator,
        steps_per_epoch=4710 // batch_size,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=522 // batch_size)

model2.save_weights('second_try_dropout_decay.h5')

In [0]:
NN_visualization(history2)

In [0]:
model_results(model2, 150)

The graphs look much better, with relatively low bias and variance, but the accuracy stalls at about 96%. The problem is more under- than overfitting at that moment. 

The test accuracy got worse, with a much lower test loss, than the previous model. 

Note: in the draft runs before the full kernel was run end to end, the results for that model were consistently much more modest, but still a significant improvement over the initial model - 81-82% test accuracy.

## Model with data augmentation

Data augmentation is a powerful tool when dealing with a small dataset. The general idea is to "augment" the images via a number of random transformations, so that the model would never see twice the exact same picture. This helps helps the model generalize better.

In Keras this can be done via the keras.preprocessing.image.ImageDataGenerator class, that I've been already using for the previous models above. The generators will need just a slight modification to add the data augmentation.

In [0]:
batch_size = 16


# The below data augmentation settings were taken from the Keras blog article in References.
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2, 
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.1)
#test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(150, 150), 
        batch_size=batch_size,
        class_mode='binary', 
        subset='training')  

validation_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(150, 150),  
        batch_size=batch_size,
        class_mode='binary', 
        subset='validation') 

In [0]:
#All settings from the previous model retained.
model3 = Sequential()
model3.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model3.add(MaxPooling2D((2, 2)))

model3.add(Conv2D(64, (3, 3), activation='relu'))
model3.add(MaxPooling2D((2, 2)))

model3.add(Conv2D(128, (3, 3), activation='relu'))
model3.add(MaxPooling2D((2, 2)))
model3.add(Dropout(0.5))

model3.add(Conv2D(128, (3, 3), activation='relu'))
model3.add(MaxPooling2D((2, 2)))
model3.add(Dropout(0.5))

model3.add(Flatten())
model3.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model3.add(Dropout(0.5))
model3.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))


model3.add(Dense(1, activation='sigmoid'))

In [0]:
model3.compile(loss='binary_crossentropy',
              optimizer=Adam(1e-4), #Retaining the very low learning ratе
              metrics=['accuracy'])

In [0]:
history3 = model3.fit_generator(
        train_generator,
        steps_per_epoch=4710 // batch_size,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=522 // batch_size)

model3.save_weights('third_try_dropout_decay_augmentation.h5')

In [0]:
NN_visualization(history3)

In [0]:
model_results(model3, 150)

WIth training and validation losses both converging and slowly decreasing, it seems like that iteration of the the model could use several more epochs. Still, 88% test accuracy is a decent result and best one up-to-now.

A note: Throughout all draft runs, this model version has been consistently the best, scoring between 91.3% and 91.6% test accuracy. The 88% achieved on that run is by far its lowest.

## Model 4 - bigger images and callbacks

While the result is already very good, compared to the benchmark, the relatively low training and validation accuracy show a room for improvement. 

I will try to improve the training accuracy by:
- increasing the image size from (150,150,3) to (224,224,3)
- implementing early stop and checkpoints via Keras callbacks

In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2, 
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.1)

train_generator_224 = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(224, 224), #increasing the image size
        batch_size=batch_size,
        class_mode='binary',
        subset='training')  


validation_generator_224 = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(224, 224),  #increasing the image size
        batch_size=batch_size,
        class_mode='binary', 
        subset='validation') 

In [0]:
model4 = Sequential()
model4.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3))) #Changing the input accordingly
model4.add(MaxPooling2D((2, 2)))

model4.add(Conv2D(64, (3, 3), activation='relu'))
model4.add(MaxPooling2D((2, 2)))

model4.add(Conv2D(128, (3, 3), activation='relu'))
model4.add(MaxPooling2D((2, 2)))
model4.add(Dropout(0.5))

model4.add(Conv2D(128, (3, 3), activation='relu'))
model4.add(MaxPooling2D((2, 2)))
model4.add(Dropout(0.5))

model4.add(Flatten())
model4.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model4.add(Dropout(0.5))
model4.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))


model4.add(Dense(1, activation='sigmoid'))

In [0]:
model4.compile(loss='binary_crossentropy',
              optimizer=Adam(1e-4), #Retaining the very low learning ratе
              metrics=['accuracy'])

In [0]:
es = EarlyStopping(patience=5) #Early Stopping will monitor the validation loss and will stop once there is not improvement for 5 steps
chkpt = ModelCheckpoint('./best_model_todate.hdf5', save_best_only=True, save_weights_only=True)


In [0]:
history4 = model4.fit_generator(
        train_generator_224,
        steps_per_epoch=4710 // batch_size,
        epochs=30, #increasing the number of epochs, considering the callbacks
        callbacks=[es, chkpt],
        validation_data=validation_generator_224,
        validation_steps=522 // batch_size)

model4.save_weights('fourth_try_dropout_decay_augmentation_callbacks.h5')

In [0]:
NN_visualization(history4)

In [0]:
model_results(model4, 224)

This iteration of the model could not improve on the previous one, despite the increased size of the image and the callbacks. Again, as with the previous version, it appears that the model could improve with more epochs or a bigger learning rate.

## Measuring success properly

As mentioned above, accuracy is not the best metrics for an imbalanced dataset. I will create the confusion matrix for all models, besides the very first one, and calculate recall and precision. 

In [0]:
test_normal = test_dir/ "NORMAL"
test_pneumonia = test_dir/ "PNEUMONIA"

#Get the true labels
true_labels = np.zeros((len(list(test_normal.rglob('*.jpeg'))),1))
true_labels = np.append(true_labels, values = np.ones((len(list(test_pneumonia.rglob('*.jpeg'))),1)))

In [0]:
def get_model_predictions(give_model, size):
  
  test_datagen = ImageDataGenerator(rescale=1./255)
  conf_matrix_generator = test_datagen.flow_from_directory(directory = test_dir,
                                  target_size=(size, size), 
                                  batch_size=1,
                                  class_mode='binary', 
                                  shuffle = False)

  predictions = give_model.predict_generator(conf_matrix_generator,624)

  prediction_rounded = []
  for i in range(len(predictions)):
    if predictions[i]>0.5:
      prediction_rounded.append(1)
    else:
      prediction_rounded.append(0)
  
  return prediction_rounded

In [0]:
cm.ravel()

In [0]:
# Get the confusion matrix
count = 2
for mdl in (model2, model3, model4):
  if mdl==model4:
    size=224
  else: size=150
  
  cm  = confusion_matrix(true_labels, get_model_predictions(mdl, size))
  plt.figure()
  plot_confusion_matrix(cm,figsize=(8,4), hide_ticks=True,cmap=plt.cm.Blues)
  plt.xticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
  plt.yticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
  plt.show()
  
  # Calculate Precision and Recall
  tn, fp, fn, tp = cm.ravel()

  precision = tp/(tp+fp)
  recall = tp/(tp+fn)
  accuracy = (tp + tn)/(tn+fp+fn+tp)
  f1 = 2 * (precision * recall) / (precision + recall)
  
  
  print("Recall of model{0} is {1:.2f}".format(count, recall))
  print("Precision of model{0} is {1:.2f}".format(count, precision))
  print("Accuracy of model{0} is {1:.2f}".format(count, accuracy))
  print("F1 score of model{0} is {1:.2f}".format(count, f1))
  print("..........................................................")
  count += 1

The best model of the 3 is model3 (Droupout and L2, but no data augmentation) with Recall of 99%, Precision of 85%. With medical topics, high recall is generally good - only a few true pneumonia cases were not captured by the model. The other two model did well in that regards too.




## Conclusion

A (relatively)small CNN and with no transfer learning achieved results comparible to the top Kaggle kernels. Perhaps smaller network is better for a binary classification and only a small dataset.

With more time and better optimization of the hyperparameters (first attempt would be adjusting the number of epochs, slightly nudging the learning rate forward and trying a different batch size), the results could most probably be improved upon.

A key lesson for myself - training the same network, especially when small, can get you 3-4% difference in accuracy.

## References:


1. Kaggle Dataset https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia/home
2. Kaggle Leading Kernel - https://www.kaggle.com/aakashnain/beating-everything-with-depthwise-convolution
3. François Chollet - https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/5.2-using-convnets-with-small-datasets.ipynb
4. Keras Blog - https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
5. Singling out this StackOverFlow - https://stackoverflow.com/questions/42443936/keras-split-train-test-set-when-using-imagedatagenerator
6. Big help on Pathlib - http://pbpython.com/pathlib-intro.*html*
7. "Deep-Learning-with-Python" - François Chollet, 2018 Manning Publications Co. ISBN 9781617294433
8. "CheXNet: Radiologist-Level Pneumonia Detection on Chest X-Rays with Deep Learning" - Rajpurkar et al. 2018
9. "Pneumonia Diagnosis using Lungs' XRays" - deadskull7 Kaggle Kernel - https://github.com/deadskull7/Pneumonia-Diagnosis-using-XRays-96-percent-Recall/blob/master/Pneumonia%20Diagnosis%20using%20Lung's%20XRay%20.ipynb



